# Converting raw .txt files from labview to fretbursts compatible HDF5 files

This notebook will convert multiple TXT files into a single combined HDF5 file, useful if you want to analyse multiple acquisitions as if they were all part of the same run.

In [1]:
import phconvert as phc
import csv
phc.__version__
import numpy as np
import os

# Read in data

Start by naming the first file and reading it in. You will need to give the length of the individual acquisitions.

In [2]:
T = 180000000000 #this is the length of EACH experiment
filenames = ["definitiveset/1c1.txt", 
            "definitiveset/1c2.txt"]
savename = 'definitiveset/1cx.hdf5'
print(filenames)


['definitiveset/1c1.txt', 'definitiveset/1c2.txt']


In [4]:
detectors = np.empty([0], dtype=int)
timestamps = np.empty([0], dtype=int)
files = 0

for file in filenames:
    with open(file) as inf:
        reader = csv.reader(inf, delimiter="	")
        ftimestamps = list(zip(*reader))[0]
    
    with open(file) as inf:
        reader = csv.reader(inf, delimiter="	")
        fdetectors = list(zip(*reader))[1]

    fdetectors = np.asarray(fdetectors)
    ftimestamps = np.asarray(ftimestamps)

    ftimestamps = np.int64(ftimestamps)
    fdetectors = np.uint8(fdetectors)
    #the following code is necessary because sometimes there are a couple of photons
    #counted after the stated end of the experiment
    bleed = 0
    for x in reversed(ftimestamps): #this is a reversed search because it's looking for numbers at the end, and if it starts at the beginning it will have to go through several million to get there
        if x > T :
            bleed +=1
        else:
            break
    ftimestamps = np.resize(ftimestamps, ftimestamps.size - bleed)
    fdetectors = np.resize(fdetectors, fdetectors.size - bleed)
    #This moves along the timestamps by which experiment it is
    ftimestamps = ftimestamps + (files * T)
    files = files + 1
    print(ftimestamps)
    timestamps = np.concatenate([timestamps, ftimestamps])
    detectors = np.concatenate([detectors, fdetectors])
    print(file + " done")
    
timestamps = np.int64(timestamps)
detectors = np.uint8(detectors)
print("all done")

[        7295        76304       113235 ... 179991641539 179991642830
 179991651103]
definitiveset/1c1.txt done
[180000301012 180000362888 180000543792 ... 359990845480 359990853519
 359990933468]
definitiveset/1c2.txt done
all done


# Create some metadata

Put information between quotation marks

In [36]:
description = '[insert description of experiment]'

author = 'Benjamin Ambrose'
author_affiliation = 'University of Sheffield'

sample_name = 'describe the sample here'
buffer_name = 'describe the buffer here'
dye_names = 'Atto550, ATTO647N'   # Comma separates names of fluorophores

# Create Photon-HDF5 data structure

In [37]:
timestamps_unit = 10e-9  # 10 ns, units are always S.I.
photon_data = dict(
    timestamps=timestamps,
    detectors=detectors,
    timestamps_specs={'timestamps_unit': timestamps_unit})
setup = dict(
    ## Mandatory fields
    num_pixels = 2,                   # using 2 detectors
    num_spots = 1,                    # a single confoca excitation
    num_spectral_ch = 2,              # donor and acceptor detection 
    num_polarization_ch = 1,          # no polarization selection 
    num_split_ch = 1,                 # no beam splitter
    modulated_excitation = False,     # CW excitation, no modulation 
    excitation_alternated = [True],  # CW excitation, no modulation 
    lifetime = False,                 # no TCSPC in detection    
    ## Optional fields
    excitation_wavelengths = [515e-9, 638e-9],         # List of excitation wavelenghts
    excitation_cw = [True],                    # List of booleans, True if wavelength is CW
    detection_wavelengths = [580e-9, 690e-9],  # Nominal center wavelength 
                                               # each for detection ch
)
identity = dict(
    author=author,
    author_affiliation=author_affiliation)
measurement_specs = dict(
    measurement_type = 'smFRET-usALEX',
    detectors_specs = {'spectral_ch1': [0],  # list of donor's detector IDs
                       'spectral_ch2': [1]},  # list of acceptor's detector IDs,
    alex_period = 10000,
    )
photon_data['measurement_specs'] = measurement_specs

data = dict(
    description=description,
    photon_data = photon_data,
    setup=setup,
    identity=identity,
)
phc.hdf5.save_photon_hdf5(data, h5_fname=savename, overwrite=True)
print("Done")